# ICLV

This demonstrates an integrated choice and latent variable in [biogeme](http://biogeme.epfl.ch), using the biogeme example data.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import biogeme.database as bdb
import biogeme.biogeme as bio
from biogeme.expressions import *
import biogeme.distributions
import biogeme.loglikelihood
import biogeme.models
import pickle

In [2]:
av = {
    0: Variable('av'),
    1: Variable('av'),
    2: Variable('av')
}

In [3]:
# estimate ICLV
# use data from biogeme
mnld = pd.read_csv('../data/optima.dat', sep='\t').sort_values('ID')
mnld['Choice'] = mnld.Choice.replace({-1: np.nan})
mnld['age'] = mnld.age.replace({-1: np.nan})
mnld['college'] = (mnld.Education >= 6).astype('int64')

mnld['incomeChf'] = mnld.Income.map({
    1: 1250,
    2: 3250,
    3: 5000,
    4: 7000,
    5: 9000,
    6: 11000,
    -1: np.nan
}) / 10000
mnld['TimeCar'] /= 60
mnld['TimePT'] /= 60
mnld['TimeWalk'] = mnld.TimeCar * 35 / 3 # assume average car trip at 35 mph, average walk at 3 mph
mnld['av'] = 1

db = bdb.Database('iclv', mnld.dropna(subset=['incomeChf', 'Choice', 'age']))

In [4]:
betas = {
    'ascCar': Beta('ascCar', 0, None, None, 0),
    'ascPt': Beta('ascPt', 0, None, None, 0),
    'travelTime': Beta('travelTime', 0, None, None, 0),
    'incCar': Beta('incCar', 0, None, None, 0),
    'incPt': Beta('incPt', 0, None, None, 0),
    
    # Latent variable influences on utility
    'envirWalk': Beta('envirWalk', 0, None, None, 0),
    'envirPt': Beta('envirPt', 0, None, None, 0),
    
    # Latent variable measurement equations
    'alphaGlobalWarming': Beta('alphaGlobalWarming', 0, None, None, 1),
    'betaGlobalWarming': Beta('betaGlobalWarming', 1, None, None, 1),
    'sigmaGlobalWarming': Beta('sigmaGlobalWarming', 1, None, None, 1),
    'alphaEconomy': Beta('alphaEconomy', 0, None, None, 0),
    'betaEconomy': Beta('betaEconomy', 0, None, None, 0),
    'sigmaEconomy': Beta('sigmaEconomy', 1, None, None, 0),
    
    # Latent variable equation
    'alphaEnvir': Beta('alphaEnvir', 0, None, None, 0),
    'ageEnvir': Beta('ageEnvir', 0, None, None, 0),
    'collegeEnvir': Beta('collegeEnvir', 0, None, None, 0),
    'envirSigma': Beta('envirSigma', 1, None, None, 0)
}

In [5]:
omega = RandomVariable('omega')
density = biogeme.distributions.normalpdf(omega)

In [6]:
# Workaround for biogeme bug: https://groups.google.com/forum/?utm_medium=email&utm_source=footer#!searchin/biogeme/bioNormalPdf%7Csort:date/biogeme/SeWFrgN74Zk/gNfM-PCsAwAJ
def bioNormalPdf(x):
    return -x*x/2 - np.log((2*np.pi) ** 0.5)
    
def loglikelihoodregression(meas,model,sigma):
    t = (meas - model) / sigma
    f = bioNormalPdf(t) - sigma
    return f

In [7]:
latentEnvironment = betas['alphaEnvir'] + betas['ageEnvir'] * Variable('age') +\
    betas['collegeEnvir'] * Variable('college') + betas['envirSigma'] * omega

globalWarming = betas['alphaGlobalWarming'] + betas['betaGlobalWarming'] * latentEnvironment
economy = betas['alphaEconomy'] + betas['betaEconomy'] * latentEnvironment

globalWarmingLikelihood = loglikelihoodregression(Variable('Envir05'), globalWarming, betas['sigmaGlobalWarming'])
economyLikelihood = loglikelihoodregression(Variable('Envir03'), economy, betas['sigmaEconomy'])

In [8]:
# 0 = pt, 1 = car, 2 = walk
utilities = {
    0: betas['ascPt'] +   betas['travelTime'] * Variable('TimePT')   + betas['incPt']  * Variable('incomeChf') + betas['envirPt'] * latentEnvironment,
    1: betas['ascCar'] +  betas['travelTime'] * Variable('TimeCar')  + betas['incCar'] * Variable('incomeChf'),
    2:                    betas['travelTime'] * Variable('TimeWalk') + betas['envirWalk'] * latentEnvironment
}

In [9]:
condprob = biogeme.models.logit(utilities, av, Variable('Choice'))
condlike = log(condprob)  + globalWarmingLikelihood + economyLikelihood
loglike = Integrate(condlike + log(density), 'omega')

In [10]:
iclv = bio.BIOGEME(db, loglike)
iclv.modelName = 'iclv'
iclvRes = iclv.estimate()

In [11]:
pd.DataFrame(iclvRes.getGeneralStatistics()).transpose()[[0]]

,0
Number of estimated parameters,14
Sample size,1699
Excluded observations,0
Init log likelihood,-3.79264e+06
Final log likelihood,-1.80211e+06
Likelihood ratio test for the init. model,3.98106e+06
Rho-square for the init. model,0.52484
Rho-square-bar for the init. model,0.524836
Akaike Information Criterion,3.60425e+06
Bayesian Information Criterion,3.60433e+06


In [12]:
res = iclvRes.getEstimatedParameters()

In [13]:
res.loc['ageEnvir',['Value', 'Std err']] *= 10 # convert to tens of years, lazily

In [14]:
res

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ageEnvir,-2.099845e-02,0.002636,-7.965959,1.554312e-15,1.680611e-03,-1.249453,2.114993e-01
alphaEconomy,5.717887e+00,0.157604,36.280093,0.000000e+00,9.416893e-01,6.071947,1.263686e-09
alphaEnvir,3.584273e+00,0.014248,251.567629,0.000000e+00,9.007843e-02,39.790577,0.000000e+00
ascCar,6.665343e+00,0.513883,12.970555,0.000000e+00,2.696063e+00,2.472250,1.342654e-02
ascPt,6.115093e+00,0.531698,11.501070,0.000000e+00,2.691105e+00,2.272335,2.306628e-02
betaEconomy,-7.951162e-01,0.043979,-18.079348,0.000000e+00,2.617903e-01,-3.037225,2.387670e-03
collegeEnvir,3.243994e-01,0.010015,32.390214,0.000000e+00,6.090093e-02,5.326673,1.000279e-07
envirPt,1.116100e-01,0.074673,1.494654,1.350047e-01,3.996101e-01,0.279297,7.800167e-01
envirSigma,-1.750216e-07,0.000469,-0.000373,9.997023e-01,3.777505e-08,-4.633258,3.599562e-06
envirWalk,1.519851e+00,0.148933,10.204963,0.000000e+00,7.786933e-01,1.951797,5.096237e-02


In [15]:
print(res[['Value', 'Std err', 't-test', 'p-value']].round(2).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  Value &  Std err &  t-test &  p-value \\
\midrule
ageEnvir     &  -0.02 &     0.00 &   -7.97 &     0.00 \\
alphaEconomy &   5.72 &     0.16 &   36.28 &     0.00 \\
alphaEnvir   &   3.58 &     0.01 &  251.57 &     0.00 \\
ascCar       &   6.67 &     0.51 &   12.97 &     0.00 \\
ascPt        &   6.12 &     0.53 &   11.50 &     0.00 \\
betaEconomy  &  -0.80 &     0.04 &  -18.08 &     0.00 \\
collegeEnvir &   0.32 &     0.01 &   32.39 &     0.00 \\
envirPt      &   0.11 &     0.07 &    1.49 &     0.14 \\
envirSigma   &  -0.00 &     0.00 &   -0.00 &     1.00 \\
envirWalk    &   1.52 &     0.15 &   10.20 &     0.00 \\
incCar       &  -1.03 &     0.09 &  -11.27 &     0.00 \\
incPt        &  -1.20 &     0.09 &  -12.64 &     0.00 \\
sigmaEconomy &   1.22 &     0.00 &  413.14 &     0.00 \\
travelTime   &  -0.60 &     0.01 &  -61.32 &     0.00 \\
\bottomrule
\end{tabular}

